# Set up Azure SQL Database for catching the bus application

This is a SQL Notebook, which allows you to separate text and code blocks and save code results. Azure Data Studio supports several languages, referred to as kernels, including SQL, PowerShell, Python, and more.

In this activity, you'll learn how to import data into Azure SQL Database and create tables to store the route data, geofence data, and real-time bus information.

## Connect to `bus-db`

At the top of the window, select **Select Connection** \> **Change Connection** next to "Attach to".

Under _Recent Connections_ select your `bus-db` connection.

You should now see it listed next to _Attach to_.

## Part 1: Import the bus route data from Azure Blob Storage

The first step in configuring the database for the scenario is to import a CSV file that contains route information data. The following script will walk you through that process. Full documentation on "Accessing data in a CSV file referencing an Azure blob storage location" here: [https://docs.microsoft.com/sql/relational-databases/import-export/examples-of-bulk-access-to-data-in-azure-blob-storage](https://docs.microsoft.com/sql/relational-databases/import-export/examples-of-bulk-access-to-data-in-azure-blob-storage).

You need to first create a table and schema for data to be loaded into.

In [1]:
CREATE TABLE [dbo].[Routes]
(
	[Id] [int] NOT NULL,
	[AgencyId] [varchar](100) NULL,
	[ShortName] [varchar](100) NULL,
	[Description] [varchar](1000) NULL,
	[Type] [int] NULL
)
GO
ALTER TABLE [dbo].[Routes] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
)
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.193

The next step is to create a master key.

In [2]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'mySuperStr0n9P@assw0rd!'

Commands completed successfully.

Total execution time: 00:00:00.046

A master key is required to create a `DATABASE SCOPED CREDENTIAL` value because Blob storage is not configured to allow public (anonymous) access. The credential refers to the Blob storage account, and the data portion specifies the container for the store return data.

We use a shared access signature as the identity that Azure SQL knows how to interpret. The secret is the SAS token that you can generate from the Blob storage account. In this example, the SAS token for a storage account that you don't have access to is provided so you can access only the store return data.

In [3]:
CREATE DATABASE SCOPED CREDENTIAL AzureBlobCredentials
WITH IDENTITY = 'SHARED ACCESS SIGNATURE',
SECRET = 'sp=r&st=2021-03-12T00:47:24Z&se=2025-03-11T07:47:24Z&spr=https&sv=2020-02-10&sr=c&sig=BmuxFevKhWgbvo%2Bj8TlLYObjbB7gbvWzQaAgvGcg50c%3D' -- Omit any leading question mark


Commands completed successfully.

Total execution time: 00:00:00.041

Next, create an external data source to the container.

In [4]:
CREATE EXTERNAL DATA SOURCE RouteData
WITH (
	TYPE = blob_storage,
	LOCATION = 'https://azuresqlworkshopsa.blob.core.windows.net/bus',
	CREDENTIAL = AzureBlobCredentials
)

Commands completed successfully.

Total execution time: 00:00:00.042

Now you are ready to bring in the data.

In [18]:
DELETE FROM dbo.[Routes];
INSERT INTO dbo.[Routes]
	([Id], [AgencyId], [ShortName], [Description], [Type])
SELECT 
	[Id], [AgencyId], [ShortName], [Description], [Type]
FROM 
OPENROWSET
	( 
		BULk 'routes.txt', 
		DATA_SOURCE = 'RouteData', 
		FORMATFILE = 'routes.fmt', 
		FORMATFILE_DATA_SOURCE = 'RouteData', 
		FIRSTROW=2,
		FORMAT='csv'
	) t;

: Msg 547, Level 16, State 0, Line 1
The DELETE statement conflicted with the REFERENCE constraint "FK__MonitoredRoutes__Router". The conflict occurred in database "bus-db", table "dbo.MonitoredRoutes", column 'RouteId'.

The statement has been terminated.

: Msg 2627, Level 14, State 1, Line 2
Violation of PRIMARY KEY constraint 'PK__Routes__3214EC07FD0F7CD6'. Cannot insert duplicate key in object 'dbo.Routes'. The duplicate key value is (100001).

The statement has been terminated.

Total execution time: 00:00:00.194

Finally, let's look at what's been inserted relative to the route we'll be tracking.

In [6]:
SELECT * FROM dbo.[Routes] WHERE [Description] LIKE '%Education Hill%'

(1 row affected)

Total execution time: 00:00:00.096

Id,AgencyId,ShortName,Description,Type
100113,1,221,Education Hill - Crossroads - Eastgate,3


## Part 2: Create necessary tables

### Select a route to monitor

Now that you've added the route information, you can select the route to be a "Monitored Route". This will come in handy if you later choose to monitor multiple routes. For now, you will just add the "Education Hill - Crossroads - Eastgate" route.

In [7]:
-- Create MonitoredRoutes table
CREATE TABLE [dbo].[MonitoredRoutes]
(
	[RouteId] [int] NOT NULL
)
GO
ALTER TABLE [dbo].[MonitoredRoutes] ADD PRIMARY KEY CLUSTERED 
(
	[RouteId] ASC
)
GO
ALTER TABLE [dbo].[MonitoredRoutes] 
ADD CONSTRAINT [FK__MonitoredRoutes__Router] 
FOREIGN KEY ([RouteId]) REFERENCES [dbo].[Routes] ([Id])
GO

-- Monitor the "Education Hill - Crossroads - Eastgate" route
INSERT INTO dbo.[MonitoredRoutes] (RouteId) VALUES (100113);

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.279

### Select a GeoFence to monitor

In addition to monitoring specific bus routes, you will want to monitor certain GeoFences so you can ultimately get notified when your bus enters or exits where you are (i.e. the GeoFence). For now, you will add a small GeoFence that represents the area near the "Crossroads" bus stop.

In [8]:
-- Create GeoFences table
CREATE SEQUENCE [dbo].[global]
    AS INT
    START WITH 1
    INCREMENT BY 1
GO
CREATE TABLE [dbo].[GeoFences](
	[Id] [int] NOT NULL,
	[Name] [nvarchar](100) NOT NULL,
	[GeoFence] [geography] NOT NULL
) 
GO
ALTER TABLE [dbo].[GeoFences] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
)
GO
ALTER TABLE [dbo].[GeoFences] ADD DEFAULT (NEXT VALUE FOR [dbo].[global]) FOR [Id]
GO
CREATE SPATIAL INDEX [ixsp] ON [dbo].[GeoFences]
(
	[GeoFence]
) USING GEOGRAPHY_AUTO_GRID 
GO

-- Create a GeoFence
INSERT INTO dbo.[GeoFences] 
	([Name], [GeoFence]) 
VALUES
	('Crossroads', 0xE6100000010407000000B4A78EA822CF4740E8D7539530895EC03837D51CEACE4740E80BFBE630895EC0ECD7DF53EACE4740E81B2C50F0885EC020389F0D03CF4740E99BD2A1F0885EC00CB8BEB203CF4740E9DB04FC23895EC068C132B920CF4740E9DB04FC23895EC0B4A78EA822CF4740E8D7539530895EC001000000020000000001000000FFFFFFFF0000000003);
GO


Commands completed successfully.

(1 row affected)

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:01.307

(No column name)
1


### Create table to track activity in the GeoFence

Next, create a system-versioned table to keep track of what activity is currently happening within the GeoFence. This means tracking buses entering, exiting, and staying within a given GeoFence. Another table within that table will serve as a histroical log for all activity.

In [9]:
CREATE TABLE [dbo].[GeoFencesActive] 
(
	[Id] [int] IDENTITY(1,1) NOT NULL PRIMARY KEY CLUSTERED,
	[VehicleId] [int] NOT NULL,
	[DirectionId] [int] NOT NULL,
	[GeoFenceId] [int] NOT NULL,
	[SysStartTime] [datetime2](7) GENERATED ALWAYS AS ROW START NOT NULL,
	[SysEndTime] [datetime2](7) GENERATED ALWAYS AS ROW END NOT NULL,
	PERIOD FOR SYSTEM_TIME ([SysStartTime], [SysEndTime])
)
WITH ( SYSTEM_VERSIONING = ON ( HISTORY_TABLE = [dbo].[GeoFencesActiveHistory] ) )
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.126

### Create a table to store real-time bus data

You'll need one last table to store the real-time bus data as it comes in.

In [10]:
CREATE TABLE [dbo].[BusData](
	[Id] [int] IDENTITY(1,1) NOT NULL,
	[DirectionId] [int] NOT NULL,
	[RouteId] [int] NOT NULL,
	[VehicleId] [int] NOT NULL,
	[Location] [geography] NOT NULL,
	[TimestampUTC] [datetime2](7) NOT NULL,
	[ReceivedAtUTC] [datetime2](7) NOT NULL
)
GO
ALTER TABLE [dbo].[BusData] ADD DEFAULT (SYSUTCDATETIME()) FOR [ReceivedAtUTC]
GO
ALTER TABLE [dbo].[BusData] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
) 
GO
CREATE NONCLUSTERED INDEX [ix1] ON [dbo].[BusData]
(
	[ReceivedAtUTC] DESC
) 
GO
CREATE SPATIAL INDEX [ixsp] ON [dbo].[BusData]
(
	[Location]
) USING GEOGRAPHY_AUTO_GRID 
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.327

Confirm you've created the tables with the following.

In [17]:
SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_SCHEMA = 'dbo'

(6 rows affected)

Total execution time: 00:00:00.064

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
bus-db,dbo,Routes,BASE TABLE
bus-db,dbo,MonitoredRoutes,BASE TABLE
bus-db,dbo,GeoFences,BASE TABLE
bus-db,dbo,GeoFencesActive,BASE TABLE
bus-db,dbo,GeoFencesActiveHistory,BASE TABLE
bus-db,dbo,BusData,BASE TABLE
